In [1]:
%load_ext autoreload
%matplotlib inline

/Users/diego/.pyenv/versions/miniconda-latest/lib/python2.7/site-packages/subprocess32.py:472: RuntimeWarning: The _posixsubprocess module is not being used. Child process reliability may suffer if your program uses threads.
  "program uses threads.", RuntimeWarning)


In [2]:
%autoreload 2

import scipy
import tensorflow as tf
from tensorflow import nn, layers
from tensorflow.contrib import layers as clayers 
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [3]:
DATA_FOLDER = './tf_data/DCGAN/CIFAR'

## Load Data

In [4]:
def load_data():
    (x_train, _), (x_test, _) = cifar10.load_data()
    # Rescale to range [0, 1]
    x = x_train.astype(float)/255 # * 2 - 1
    # Shuffle
    np.random.shuffle(x)
    return x

data = load_data()
data.shape

(50000, 32, 32, 3)

In [5]:
IMAGES_SHAPE = (64, 64, 3)

## Initialize Graph

In [6]:
def default_conv2d(input, filters):
    return layers.conv2d(
        input,
        filters=filters,
        kernel_size=4,
        strides=(2, 2),
        padding='valid',
        data_format='channels_last',
        use_bias=False
    )

#### Discriminator

In [7]:
## Input
X = tf.placeholder(tf.float32, shape=(None, )+IMAGES_SHAPE)

def discriminator(x):
    with tf.variable_scope("d"):
        with tf.variable_scope("conv1"):
            conv1 = default_conv2d(x, 128)
            conv1 = nn.leaky_relu(conv1,alpha=0.2)
        
        with tf.variable_scope("conv2"):
            conv2 = default_conv2d(conv1, 256)
            conv2 = layers.batch_normalization(conv2)
            conv2 = nn.leaky_relu(conv2,alpha=0.2)
            
        with tf.variable_scope("conv3"):
            conv3 = default_conv2d(conv2, 512)
            conv3 = layers.batch_normalization(conv3)
            conv3 = nn.leaky_relu(conv3,alpha=0.2)
            
        with tf.variable_scope("conv4"):
            conv4 = default_conv2d(conv3, 1024)
            conv4 = layers.batch_normalization(conv3)
            conv4 = nn.leaky_relu(conv3,alpha=0.2)
        
        with tf.variable_scope("linear"):
            linear = clayers.flatten(conv4)
            linear = clayers.fully_connected(linear, 1)
        
        with tf.variable_scope("out"):
            out = nn.sigmoid(linear)
    return out

<tf.Tensor 'd/linear/Sigmoid:0' shape=(?, 1) dtype=float32>